<a href="https://colab.research.google.com/github/SagarSharma250/Chatbot/blob/main/Chatbot_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [65]:
# !pip install langchain
# !pip install pypdf
# !pip install cohere
# !pip install qdrant-client
# !pip install gradio

In [25]:
#Import Libraries
import os
import langchain
import gradio as gr

In [26]:
import warnings
# Filter out the LangChainDeprecationWarning
warnings.filterwarnings("ignore")

In [27]:
from google.colab import drive
drive.mount("/content/gdrive")
os.chdir('/content/gdrive/MyDrive/Colab/Chatbot')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [28]:
#Load content from PDF Document
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader ("./llm.pdf")
#Provide the respective file path of the PDF Document
documents = loader.load()

In [31]:
documents

[Document(page_content="Large\nlanguage\nmodels\n(LLMs)\nrepresent\na\ngroundbreaking\nadvancement\nin\nartificial\nintelligence,\nparticularly\nin\nnatural\nlanguage\nprocessing.\nThese\nmodels,\nlike\nOpenAI's\nGPT\n(Generative\nPre-trained\nTransformer)\nseries,\nare\ncharacterized\nby\ntheir\nimmense\nsize\nand\ncomplexity,\ntypically\ncomprising\nbillions\nor\neven\ntrillions\nof\nparameters.\nBy\nleveraging\nvast\namounts\nof\ntext\ndata\nfrom\nthe\ninternet,\nLLMs\nare\npre-trained\non\na\ndiverse\nrange\nof\nlinguistic\ntasks,\nenabling\nthem\nto\nunderstand\nand\ngenerate\nhuman-like\ntext\nacross\nvarious\ndomains\nand\nlanguages.\nThis\npre-training\nphase\nis\ncrucial\nas\nit\nallows\nthe\nmodel\nto\nlearn\nintricate\npatterns\nand\nnuances\nof\nlanguage,\neffectively\ncapturing\nsemantic\nrelationships\nand\ncontextual\ninformation.\nOne\nof\nthe\nkey\nstrengths\nof\nLLMs\nlies\nin\ntheir\nability\nto\nperform\na\nwide\narray\nof\nnatural\nlanguage\nprocessing\ntasks\nwith

In [7]:
#Print PDF Document Content
#Based on the Volume of the Content of the Document we can print all documents or few extracts
# doc = documents[0]
# page_content = doc.page_content[:10000]
# print(page_content[:2000])

In [32]:
#Split the Document into chunks for embedding and vector storage.
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 50)
all_splits = text_splitter.split_documents(documents)

In [33]:
print(all_splits)

[Document(page_content="Large\nlanguage\nmodels\n(LLMs)\nrepresent\na\ngroundbreaking\nadvancement\nin\nartificial\nintelligence,\nparticularly\nin\nnatural\nlanguage\nprocessing.\nThese\nmodels,\nlike\nOpenAI's\nGPT\n(Generative\nPre-trained\nTransformer)\nseries,\nare\ncharacterized\nby\ntheir\nimmense\nsize\nand\ncomplexity,\ntypically\ncomprising\nbillions\nor\neven\ntrillions\nof\nparameters.\nBy\nleveraging\nvast\namounts\nof\ntext\ndata\nfrom\nthe\ninternet,\nLLMs\nare\npre-trained\non\na\ndiverse\nrange\nof\nlinguistic\ntasks,\nenabling\nthem\nto\nunderstand\nand\ngenerate", metadata={'source': './llm.pdf', 'page': 0}), Document(page_content="tasks,\nenabling\nthem\nto\nunderstand\nand\ngenerate\nhuman-like\ntext\nacross\nvarious\ndomains\nand\nlanguages.\nThis\npre-training\nphase\nis\ncrucial\nas\nit\nallows\nthe\nmodel\nto\nlearn\nintricate\npatterns\nand\nnuances\nof\nlanguage,\neffectively\ncapturing\nsemantic\nrelationships\nand\ncontextual\ninformation.\nOne\nof\nthe\nke

In [34]:
import os
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.vectorstores import Qdrant
os.environ['COHERE_API_KEY'] =  '***sdlan***'    #Set the Cohere API Key
embeddings = CohereEmbeddings(model = "multilingual-22-12")
db = Qdrant.from_documents(all_splits, embeddings, location=":memory:", collection_name="all_splits", distance_func="Dot")

In [39]:
import langchain
langchain.debug = False
from langchain.chains import RetrievalQA
from langchain.llms import Cohere
#temperature defines randomness of the output 0 defines least randomness 1 defines maximum randomness
qa = RetrievalQA.from_chain_type(llm=Cohere(model="command-nightly",temperature=0.5),
                                 chain_type="stuff",
                                 retriever=db.as_retriever(),
                                 return_source_documents=True)

In [40]:
questions = [
           "How are language Translators job redefined ?",
           "What did the article say about Sam?",
           "What did the article say about Covid?"
           ]

In [41]:
for question in questions:
  answer = qa({"query": question})
  print(answer)

{'query': 'How are language Translators job redefined ?', 'result': "I don't know.", 'source_documents': [Document(page_content="tasks,\nenabling\nthem\nto\nunderstand\nand\ngenerate\nhuman-like\ntext\nacross\nvarious\ndomains\nand\nlanguages.\nThis\npre-training\nphase\nis\ncrucial\nas\nit\nallows\nthe\nmodel\nto\nlearn\nintricate\npatterns\nand\nnuances\nof\nlanguage,\neffectively\ncapturing\nsemantic\nrelationships\nand\ncontextual\ninformation.\nOne\nof\nthe\nkey\nstrengths\nof\nLLMs\nlies\nin\ntheir\nability\nto\nperform\na\nwide\narray\nof\nnatural\nlanguage\nprocessing\ntasks\nwith\nminimal\nfine-tuning.\nWhether\nit's\ntext\ngeneration,\nlanguage\ntranslation,\nsentiment\nanalysis,", metadata={'source': './llm.pdf', 'page': 0, '_id': '817dedfc46184517b2b98b596212b4e3', '_collection_name': 'all_splits'}), Document(page_content='language\ntranslation,\nsentiment\nanalysis,\nor\nquestion\nanswering,\nthese\nmodels\ndemonstrate\nremarkable\nversatility\nand\nadaptability.\nMoreover

In [43]:
from langchain.llms import Cohere
llm = Cohere(model = "command", temperature=0.5)

In [44]:
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.conversation.memory import ConversationSummaryMemory
#Setup Chat History
#The chat_history variable keeps track of the conversation history,
#storing the user queries #and the chatbot's responses
#This allows the chatbot to maintain context and provide relevant answers
# based on past interactions.
chat_history = []
qa = ConversationalRetrievalChain.from_llm(
    llm = llm,
    chain_type = "stuff",
    memory = ConversationSummaryMemory(llm = llm, memory_key='chat_history', input_key='question', output_key= 'answer', return_messages=True),
    retriever = db.as_retriever(),
    return_source_documents=False
)

Implementation using Gardio Interface

In [52]:
import gradio as gr
import sys

# Define the function to update user info
user_info = {'data': []}
update_info_result = ''

def update_user_info(name, phone_number, email):
    global update_info_result
    user_info['data'].append({
        'name': name,
        'phone_number': phone_number,
        'email': email
    })
    update_info_result = "User info updated successfully."

# Define the function for Document QA
def QandA(question):
    global chat_history
    result = qa({"question": question, "chat_history": chat_history})
    chat_history.append((question, result['answer']))
    return result['answer']

# Define the Gradio interface for updating user info
def update_info(name, phone_number, email):
    update_user_info(name, phone_number, email)
    return "User info updated successfully."

UpdateInfoUI = gr.Interface(update_info,
                             inputs=["text", "text", "text"],
                             outputs="text",
                             title="Update User Info",
                             description="Enter your name, phone number, and email to update user information.")

# Define the Gradio interface for chatting with the chatbot
def chat_interface():
    return gr.Interface(QandA,
                        inputs="text",
                        outputs="text",
                        title="Chat with Chatbot",
                        description="Ask your questions here.")

# Launch the Gradio interfaces
update_info_result = UpdateInfoUI.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://dd89a2477ca1f9f0fa.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [46]:
# Check if user info was updated successfully
if update_info_result == "User info updated successfully.":
    # Close the "Update User Info" interface
    UpdateInfoUI.close()
    # Launch the interface for chatting with the chatbot
    chat_interface().launch(share=True)

# Initialize chat history
chat_history = []

Closing server running on port: 7862
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://d3683ce47f8d93c1ee.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Notebook implementation of User form and chatbot query

In [62]:
import sys

# Define the function to update user info
user_info = {'data': []}
update_info_result = ''

def update_user_info(name, phone_number, email):
    global update_info_result
    user_info['data'].append({
        'name': name,
        'phone_number': phone_number,
        'email': email
    })
    update_info_result = "User info updated successfully."

# Define the function for Document QA
def QandA(question):
    global chat_history
    result = qa({"question": question, "chat_history": chat_history})
    chat_history.append((question, result['answer']))
    return result['answer']

# Define the Gradio interface for chatting with the chatbot
def chat_interface():
    if update_info_result !="User info updated successfully.":
      name = input("Your Name: ")
      phone_number = input("Your Phone Number: ")
      email = input("Your Email: ")
      update_user_info(name,phone_number,email)
    while(True):
        print("Enter your query to the chatbot")
        question = input("Your Query:")
        if(question=='quit'):
          break
        else:
          answer=QandA(question)
          print(answer)

In [63]:
chat_interface()

Your Name: ss
Your Phone Number: 9865875
Your Email: ss@gmail.com
Enter your query to the chatbot
Your Query:explain about deployment of large language models?
 The deployment of LLMs raises several key ethical and societal concerns. Here are three examples of these concerns: 

1. Environmental impact: One significant concern is the astronomical environmental impact of LLMs. These models require immense computational resources to train and operate, leading to substantial energy consumption and carbon emissions. This is especially problematic in the current climate crisis, as AI technologies contribute to greenhouse gas emissions and the overall climate impact of the tech sector. 

2. Bias in training data: Another key concern is bias in the training data that is used to teach the LLMs. If the data is biased, the models themselves can reflect and even amplify such biases, leading to discriminatory or unfair outputs. This can have serious societal consequences, especially in sensitive do

 Large Language Models (LLMs) have a number of benefits, but one of the key advantages is their ability to perform a wide array of language processing tasks with minimal fine-tuning. This means that these models can understand and generate human-like text across various domains and languages with ease. They can be applied to text generation, language translation, sentiment analysis, and question answering tasks, demonstrating remarkable versatility and adaptability. 

This advantage arises from the pre-training phase of these models, which allows them to learn intricate patterns and nuances of language. effectively capturing semantic relationships and contextual information. 
Enter your query to the chatbot
Your Query:quit


In [64]:
user_info

{'data': [{'name': 'ss', 'phone_number': '9865875', 'email': 'ss@gmail.com'}]}